Generated By Chatgpt + Tweaked By: Surya

In [1]:
from manim import *
import numpy as np

# ------------------------------
# Helper Classes
# ------------------------------

class Arrow3DSafe(Arrow3D):
    """Stable Arrow3D wrapper for Manim v0.19"""
    def __init__(self, start=ORIGIN, end=RIGHT, color=WHITE, **kwargs):
        super().__init__(
            start=start,
            end=end,
            color=color,
            **{k: v for k,v in kwargs.items() if k not in ["tip_length","thickness","scale_tips"]}
        )

class MatrixTransformHelper:
    @staticmethod
    def apply_matrix_to_vector(M, v):
        v = np.asarray(v).reshape(3,)
        return M.dot(v)


# ------------------------------
# Main Scene
# ------------------------------

class Covariance3D(ThreeDScene):
    def construct(self):
        # Parameters
        mu = np.array([0.5, -0.2, 0.3])
        Sigma = np.array([[1.0, 0.6, 0.2],
                          [0.6, 0.8, 0.1],
                          [0.2, 0.1, 0.5]])
        n_samples = 600
        short, medium, long = 0.8, 1.4, 2.2

        # Eigen decomposition
        eigvals, eigvecs = np.linalg.eigh(Sigma)
        order = np.argsort(eigvals)[::-1]
        eigvals = eigvals[order]
        eigvecs = eigvecs[:, order]
        S = eigvecs.dot(np.diag(np.sqrt(eigvals)))

        # Axes
        axes = ThreeDAxes(x_range=[-3,3,1], y_range=[-3,3,1], z_range=[-3,3,1])
        self.add(axes)
        for direction, axis in zip([RIGHT, UP, OUT], ['x','y','z']):
            label = MathTex(axis, font_size=24)
            label.next_to(3*direction, direction)
            self.add_fixed_orientation_mobjects(label)

        self.set_camera_orientation(phi=60*DEGREES, theta=-45*DEGREES, distance=12)

        # Unit Cube + Transformed Cube
        cube = Cube(side_length=1, fill_opacity=0.1, stroke_width=1).scale(1.2)
        transformed_cube = cube.copy()
        def apply_linear_to_mobject(mobj, matrix, offset=np.zeros(3)):
            points = mobj.get_points()
            new_points = np.dot(points, matrix.T) + offset
            mobj.set_points(new_points)

        lbl_cube = Tex("Unit cube", font_size=24).next_to(cube, DOWN)
        lbl_tcube = Tex("Transformed cube", font_size=24).next_to(cube, DOWN).shift(RIGHT*3)

        cube.to_edge(LEFT*2)
        transformed_cube.to_edge(RIGHT*2)
        transformed_cube_center = transformed_cube.get_center()
        transformed_cube.shift(-transformed_cube_center)
        apply_linear_to_mobject(transformed_cube, S)
        transformed_cube.shift(transformed_cube_center + mu)

        self.play(FadeIn(cube), Write(lbl_cube), run_time=short)
        self.play(FadeIn(transformed_cube), Write(lbl_tcube), run_time=short)
        self.wait(0.8)

        # Basis vectors
        basis = VGroup()
        basis_transformed = VGroup()
        colors = [RED, GREEN, BLUE]
        for i, vec in enumerate([RIGHT, UP, OUT]):
            arr = Arrow3DSafe(start=ORIGIN, end=vec*1.5, color=colors[i])
            basis.add(arr)
            tvec = MatrixTransformHelper.apply_matrix_to_vector(S, np.array(vec))
            arr_t = Arrow3DSafe(start=mu, end=mu+tvec*1.5, color=colors[i])
            basis_transformed.add(arr_t)

        b_labels = VGroup(*[MathTex(f"e_{i+1}") for i in range(3)])
        for i, label in enumerate(b_labels):
            label.next_to(basis[i].get_end(), UP*0.3)

        # Use Create instead of GrowArrow
        self.play(LaggedStartMap(Create, basis, lag_ratio=0.2), run_time=medium)
        self.play(LaggedStartMap(FadeIn, b_labels, lag_ratio=0.15), run_time=short)
        self.wait(0.6)

        self.play(
            LaggedStart(*[Transform(basis[i], basis_transformed[i]) for i in range(3)], lag_ratio=0.2),
            run_time=long
        )
        self.wait(0.6)

        # Gaussian Cloud
        rng = np.random.default_rng(seed=42)
        samples = rng.multivariate_normal(mean=mu, cov=Sigma, size=n_samples)
        dots = VGroup()
        for p in samples:
            s = Sphere(radius=0.03)
            s.move_to(p)
            s.set_fill(opacity=0.9)
            s.set_stroke(width=0.02)
            dots.add(s)
        self.play(LaggedStartMap(FadeIn, dots, lag_ratio=0.01), run_time=2.5)
        self.wait(0.6)

        # Covariance Ellipsoid
        def ellipsoid(u,v):
            x = np.sin(u)*np.cos(v)
            y = np.sin(u)*np.sin(v)
            z = np.cos(u)
            return np.dot(S, np.array([x,y,z])) + mu

        ellipsoid_surface = Surface(lambda u,v: ellipsoid(u,v),
                                    u_range=[0, PI], v_range=[0, TAU],
                                    resolution=(24,24),
                                    fill_opacity=0.25,
                                    checkerboard_colors=[BLUE_E, BLUE_D])
        self.play(FadeIn(ellipsoid_surface), run_time=1.6)
        self.wait(0.6)

        # Eigenvectors
        eig_arrows = VGroup()
        eig_labels = VGroup()
        for i in range(3):
            vec = eigvecs[:, i]
            length = np.sqrt(eigvals[i])*1.6
            arr = Arrow3DSafe(start=mu-vec*length, end=mu+vec*length, color=colors[i])
            eig_arrows.add(arr)
            label = MathTex(f"\\lambda_{i+1}={eigvals[i]:.2f}")
            label.next_to(arr.get_end(), RIGHT*0.2)
            eig_labels.add(label)

        self.play(LaggedStartMap(Create, eig_arrows, lag_ratio=0.2), run_time=short)
        self.play(LaggedStartMap(FadeIn, eig_labels, lag_ratio=0.15), run_time=short)
        self.wait(0.6)

        # Covariance matrices
        sigma_tex = Matrix([[f"{Sigma[i,j]:.2f}" for j in range(3)] for i in range(3)])
        sigma_tex.scale(0.7)
        sigma_tex.to_corner(UL)
        decomp_tex = VGroup(MathTex("\\Sigma = U \\Lambda U^T", font_size=28),
                            Matrix([[f"{eigvals[i]:.2f}" if i==j else "0.00" for j in range(3)] for i in range(3)]).scale(0.6))
        decomp_tex.arrange(DOWN, buff=0.15)
        decomp_tex.next_to(sigma_tex, DOWN)
        self.play(Write(sigma_tex), run_time=short)
        self.play(FadeIn(decomp_tex), run_time=short)
        self.wait(0.8)

        # Camera motion
        self.begin_ambient_camera_rotation(rate=0.12)
        self.wait(3)
        self.stop_ambient_camera_rotation()
        self.move_camera(phi=65*DEGREES, theta=45*DEGREES, frame_center=ORIGIN, run_time=2)
        self.wait(1)

        # Legend
        legend = VGroup(MathTex("\\text{Cloud (samples)}"),
                        MathTex("\\text{Ellipsoid (covariance)}"),
                        MathTex("\\text{Eigenvectors / principal axes}"))
        legend.arrange(DOWN, aligned_edge=LEFT)
        legend.scale(0.6)
        legend.to_corner(DL)
        self.play(FadeIn(legend), run_time=short)
        self.wait(2)
        self.play(FadeOut(legend), FadeOut(sigma_tex), FadeOut(decomp_tex), FadeOut(lbl_cube), FadeOut(lbl_tcube))
        self.wait(0.8)


%manim -ql -v warning Covariance3D

Manim Community v0.19.0

/tmp/ipykernel_82890/2765335503.py:60: DeprecationWarning: This method is not guaranteed to stay around. Please prefer getting the attribute normally.
  points = mobj.get_points()


[11/12/25 11:47:48] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=936829;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=698815;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=272963;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=305498;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

[11/12/25 12:12:25] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=839715;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=441803;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

...

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=79074;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=828507;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

[11/12/25 13:17:36] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=29641;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=898113;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=304098;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=141136;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

[11/12/25 13:29:45] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=865997;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=600458;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=151946;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=61202;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

In [1]:
from manim import *
import numpy as np

# ------------------------------
# Helper Classes (Unchanged)
# ------------------------------

class Arrow3DSafe(Arrow3D):
    """Stable Arrow3D wrapper for Manim v0.19"""
    def __init__(self, start=ORIGIN, end=RIGHT, color=WHITE, **kwargs):
        super().__init__(
            start=start,
            end=end,
            color=color,
            **{k: v for k,v in kwargs.items() if k not in ["tip_length","thickness","scale_tips"]}
        )

class MatrixTransformHelper:
    @staticmethod
    def apply_matrix_to_vector(M, v):
        v = np.asarray(v).reshape(3,)
        return M.dot(v)


# ------------------------------
# Main Scene (Refactored)
# ------------------------------

class Covariance3D(ThreeDScene):
    def construct(self):
        # Parameters
        mu = np.array([0.5, -0.2, 0.3])
        Sigma = np.array([[1.0, 0.6, 0.2],
                          [0.6, 0.8, 0.1],
                          [0.2, 0.1, 0.5]])
        n_samples = 200  # <-- Reduced for speed
        short, medium, long = 0.5, 1.0, 1.5 # <-- Reduced for speed

        # Eigen decomposition
        eigvals, eigvecs = np.linalg.eigh(Sigma)
        order = np.argsort(eigvals)[::-1]
        eigvals = eigvals[order]
        eigvecs = eigvecs[:, order]
        S = eigvecs.dot(np.diag(np.sqrt(eigvals)))

        # Axes
        axes = ThreeDAxes(x_range=[-3,3,1], y_range=[-3,3,1], z_range=[-3,3,1], x_length=6, y_length=6, z_length=6)
        axes.scale(0.8) # Scale down axes to make room for text
        self.add(axes)
        for direction, axis in zip([RIGHT, UP, OUT], ['x','y','z']):
            label = MathTex(axis, font_size=24)
            label.next_to(axes.get_axis(np.argmax(direction)).get_end(), direction*0.5)
            self.add_fixed_orientation_mobjects(label)

        self.set_camera_orientation(phi=60*DEGREES, theta=-45*DEGREES, distance=12)
        
        # --- Right-Side Text Panel ---
        # This VGroup will hold all our text
        text_panel = VGroup()
        text_panel.to_corner(UR, buff=0.5)

        def add_text_step(title_text, body_text):
            title = Tex(title_text, font_size=28, color=YELLOW)
            body = Tex(body_text, font_size=24, color=WHITE)
            step_group = VGroup(title, body).arrange(DOWN, aligned_edge=LEFT, buff=0.2)
            
            if len(text_panel) > 0:
                step_group.next_to(text_panel, DOWN, aligned_edge=LEFT, buff=0.4)
            
            text_panel.add(step_group)
            self.add_fixed_orientation_mobjects(step_group)
            return FadeIn(step_group)

        # --- Scene Animations ---

        # 1. Unit Cube + Transformed Cube
        cube = Cube(side_length=1, fill_opacity=0.1, stroke_width=1).scale(1.2)
        transformed_cube = cube.copy()
        
        def apply_linear_to_mobject(mobj, matrix, offset=np.zeros(3)):
            points = mobj.get_points()
            new_points = np.dot(points, matrix.T) + offset
            mobj.set_points(new_points)

        # Move cubes to the left to make space for text
        cube.to_edge(LEFT, buff=2)
        transformed_cube.to_edge(LEFT, buff=2).shift(RIGHT*3)
        
        transformed_cube_center = transformed_cube.get_center()
        transformed_cube.shift(-transformed_cube_center)
        apply_linear_to_mobject(transformed_cube, S)
        transformed_cube.shift(transformed_cube_center + mu)

        self.play(
            FadeIn(cube),
            FadeIn(transformed_cube),
            add_text_step("1. Linear Transformation", "Unit cube transformed by $S$ \nand shifted by $\\mu$."),
            run_time=medium
        )
        self.wait(short)

        # 2. Basis vectors
        basis = VGroup()
        basis_transformed = VGroup()
        colors = [RED, GREEN, BLUE]
        for i, vec in enumerate([RIGHT, UP, OUT]):
            arr = Arrow3DSafe(start=ORIGIN, end=vec*1.5, color=colors[i])
            basis.add(arr)
            tvec = MatrixTransformHelper.apply_matrix_to_vector(S, np.array(vec))
            arr_t = Arrow3DSafe(start=mu, end=mu+tvec*1.5, color=colors[i])
            basis_transformed.add(arr_t)

        self.play(
            LaggedStartMap(Create, basis, lag_ratio=0.2),
            add_text_step("2. Basis Vectors", "Standard basis $e_1, e_2, e_3$."),
            run_time=medium
        )
        self.wait(short)

        self.play(
            LaggedStart(*[Transform(basis[i], basis_transformed[i]) for i in range(3)], lag_ratio=0.2),
            add_text_step("", "Transformed to $S \\cdot e_i + \\mu$."), # No title for continuation
            run_time=long
        )
        self.wait(short)

        # 3. Gaussian Cloud
        rng = np.random.default_rng(seed=42)
        samples = rng.multivariate_normal(mean=mu, cov=Sigma, size=n_samples)
        dots = VGroup()
        for p in samples:
            s = Sphere(radius=0.03)
            s.move_to(p)
            s.set_fill(opacity=0.9)
            s.set_stroke(width=0.02)
            dots.add(s)
            
        self.play(
            FadeIn(dots, run_time=long), # Use simple FadeIn for speed
            add_text_step("3. Data Cloud", f"{n_samples} samples drawn from \n$N(\\mu, \\Sigma)$."),
            run_time=long
        )
        self.wait(short)

        # 4. Covariance Ellipsoid
        def ellipsoid_func(u,v):
            x = np.sin(u)*np.cos(v)
            y = np.sin(u)*np.sin(v)
            z = np.cos(u)
            return np.dot(S, np.array([x,y,z])) + mu

        ellipsoid_surface = Surface(lambda u,v: ellipsoid_func(u,v),
                                    u_range=[0, PI], v_range=[0, TAU],
                                    resolution=(24,24),
                                    fill_opacity=0.25,
                                    checkerboard_colors=[BLUE_E, BLUE_D])
        
        self.play(
            FadeIn(ellipsoid_surface),
            add_text_step("4. Covariance Ellipsoid", "Shows the 1-standard-deviation \nsurface of the distribution."),
            run_time=medium
        )
        self.wait(short)

        # 5. Eigenvectors
        eig_arrows = VGroup()
        for i in range(3):
            vec = eigvecs[:, i]
            length = np.sqrt(eigvals[i])*1.6
            arr = Arrow3DSafe(start=mu-vec*length, end=mu+vec*length, color=colors[i])
            eig_arrows.add(arr)

        self.play(
            LaggedStartMap(Create, eig_arrows, lag_ratio=0.2),
            add_text_step("5. Principal Axes", "Eigenvectors of $\\Sigma$, showing \ndirections of max variance."),
            run_time=medium
        )
        self.wait(short)

        # 6. Covariance matrices
        sigma_tex = VGroup(
            Tex("Covariance Matrix $\\Sigma$:", font_size=24, color=YELLOW),
            Matrix([[f"{Sigma[i,j]:.2f}" for j in range(3)] for i in range(3)], h_buff=1.5).scale(0.6)
        ).arrange(DOWN, buff=0.15)
        
        decomp_tex = VGroup(
            Tex("Eigenvalues (Variance):", font_size=24, color=YELLOW),
            MathTex(f"\\lambda_1 = {eigvals[0]:.2f}", f"\\lambda_2 = {eigvals[1]:.2f}", f"\\lambda_3 = {eigvals[2]:.2f}", font_size=24)
        ).arrange(DOWN, aligned_edge=LEFT, buff=0.15)
        
        matrices_group = VGroup(sigma_tex, decomp_tex).arrange(DOWN, buff=0.4)
        matrices_group.next_to(text_panel, DOWN, aligned_edge=LEFT, buff=0.4)
        
        self.add_fixed_orientation_mobjects(matrices_group)
        self.play(FadeIn(matrices_group), run_time=medium)
        text_panel.add(matrices_group) # Add to panel for fading out
        self.wait(medium)

        # 7. Camera motion
        self.begin_ambient_camera_rotation(rate=0.1)
        self.wait(3)
        self.stop_ambient_camera_rotation()
        self.move_camera(phi=65*DEGREES, theta=45*DEGREES, frame_center=ORIGIN, run_time=long)
        self.wait(medium)

        # 8. Fade out
        self.play(
            FadeOut(dots),
            FadeOut(ellipsoid_surface),
            FadeOut(eig_arrows),
            FadeOut(basis),
            FadeOut(cube),
            FadeOut(transformed_cube),
            FadeOut(text_panel), # Fade out the entire text panel
            run_time=medium
        )
        self.wait(short)

%manim -ql -v warning Covariance3D

Manim Community v0.19.0

/tmp/ipykernel_9444/63069307.py:81: DeprecationWarning: This method is not guaranteed to stay around. Please prefer getting the attribute normally.
  points = mobj.get_points()


[11/12/25 18:59:02] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=294892;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=922859;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=410736;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=405360;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

[11/12/25 19:03:44] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=144181;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=356331;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

...

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=520750;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=871452;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

[11/12/25 19:21:05] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=700244;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=562521;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=402795;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=227037;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

[11/12/25 19:21:39] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=903314;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=396808;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=180132;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=471612;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

In [4]:
from manim import *
import numpy as np

# ------------------------------
# Helper Classes (Unchanged)
# ------------------------------

class Arrow3DSafe(Arrow3D):
    """Stable Arrow3D wrapper for Manim v0.19"""
    def __init__(self, start=ORIGIN, end=RIGHT, color=WHITE, **kwargs):
        super().__init__(
            start=start,
            end=end,
            color=color,
            **{k: v for k,v in kwargs.items() if k not in ["tip_length","thickness","scale_tips"]}
        )

class MatrixTransformHelper:
    @staticmethod
    def apply_matrix_to_vector(M, v):
        v = np.asarray(v).reshape(3,)
        return M.dot(v)


# ------------------------------
# Main Scene (Refactored)
# ------------------------------

class Covariance3D(ThreeDScene):
    def construct(self):
        # Parameters
        mu = np.array([0.5, -0.2, 0.3])
        Sigma = np.array([[1.0, 0.6, 0.2],
                          [0.6, 0.8, 0.1],
                          [0.2, 0.1, 0.5]])
        n_samples = 200  # Reduced for speed
        short, medium, long = 0.5, 1.0, 1.5 # Reduced for speed

        # Eigen decomposition
        eigvals, eigvecs = np.linalg.eigh(Sigma)
        order = np.argsort(eigvals)[::-1]
        eigvals = eigvals[order]
        eigvecs = eigvecs[:, order]
        S = eigvecs.dot(np.diag(np.sqrt(eigvals)))

        # Axes
        axes = ThreeDAxes(x_range=[-3,3,1], y_range=[-3,3,1], z_range=[-3,3,1], x_length=6, y_length=6, z_length=6)
        axes.scale(0.8) # Scale down axes to make room for text
        self.add(axes)
        for direction, axis in zip([RIGHT, UP, OUT], ['x','y','z']):
            label = MathTex(axis, font_size=24)
            label.next_to(axes.get_axis(np.argmax(direction)).get_end(), direction*0.5)
            self.add_fixed_orientation_mobjects(label)

        self.set_camera_orientation(phi=60*DEGREES, theta=-45*DEGREES, distance=12)
        
        # --- Right-Side Text Panel Helper ---
        # This VGroup will hold our current text step
        current_text_group = VGroup()

        def display_text_step(title_text, body_text):
            nonlocal current_text_group
            
            # 1. Create the new text mobject
            title = Tex(title_text, font_size=28, color=YELLOW)
            body = Tex(body_text, font_size=24, color=WHITE)
            new_text_group = VGroup(title, body).arrange(DOWN, aligned_edge=LEFT, buff=0.2)
            new_text_group.to_corner(UR, buff=0.5)
            
            self.add_fixed_orientation_mobjects(new_text_group)
            
            # 2. Create the list of animations to play
            anims_to_play = [FadeIn(new_text_group)]
            
            # 3. If old text exists, add its FadeOut animation
            if current_text_group:
                anims_to_play.append(FadeOut(current_text_group))
                
            # 4. Update the global reference
            current_text_group = new_text_group
            
            # 5. Return the list of animations
            return anims_to_play


        # --- Scene Animations ---

        # 1. Display Mean Vector
        mean_arrow = Arrow3DSafe(start=ORIGIN, end=mu, color=YELLOW_D, thickness=0.08)
        text_anims = display_text_step("1. Mean Vector ($\\mu$)", "The center of the data distribution.")
        self.play(
            Create(mean_arrow),
            *text_anims,
            run_time=medium
        )
        self.wait(short)

        # 2. Gaussian Cloud
        rng = np.random.default_rng(seed=42)
        samples = rng.multivariate_normal(mean=mu, cov=Sigma, size=n_samples)
        dots = VGroup()
        for p in samples:
            s = Sphere(radius=0.03)
            s.move_to(p)
            s.set_fill(opacity=0.9)
            s.set_stroke(width=0.02)
            dots.add(s)
            
        text_anims = display_text_step("2. Data Cloud (Samples)", f"{n_samples} samples drawn from a \nmultivariate Gaussian distribution.")
        self.play(
            FadeIn(dots),
            *text_anims,
            run_time=long
        )
        self.wait(short)

        # 3. Covariance Ellipsoid
        def ellipsoid_func(u,v):
            x = np.sin(u)*np.cos(v)
            y = np.sin(u)*np.sin(v)
            z = np.cos(u)
            return np.dot(S, np.array([x,y,z])) + mu

        ellipsoid_surface = Surface(lambda u,v: ellipsoid_func(u,v),
                                    u_range=[0, PI], v_range=[0, TAU],
                                    resolution=(24,24),
                                    fill_opacity=0.25,
                                    checkerboard_colors=[BLUE_E, BLUE_D])
        
        text_anims = display_text_step("3. Covariance Ellipsoid", "Represents the 1-standard-deviation \nsurface of the distribution's shape.")
        self.play(
            FadeIn(ellipsoid_surface),
            *text_anims,
            run_time=medium
        )
        self.wait(short)

        # 4. Eigenvectors (Principal Axes)
        eig_arrows = VGroup()
        colors = [RED, GREEN, BLUE]
        for i in range(3):
            vec = eigvecs[:, i]
            length = np.sqrt(eigvals[i])*1.6 # Scale by sqrt(eigenvalue)
            arr = Arrow3DSafe(start=mu-vec*length, end=mu+vec*length, color=colors[i], thickness=0.06)
            eig_arrows.add(arr)

        text_anims = display_text_step("4. Principal Axes (Eigenvectors)", "Directions of maximum variance in the data. \nLengths are proportional to $\\sqrt{\\lambda_i}$.")
        self.play(
            LaggedStartMap(Create, eig_arrows, lag_ratio=0.2),
            *text_anims,
            run_time=medium
        )
        self.wait(short)

        # 5. Covariance Matrix
        sigma_tex = VGroup(
            Tex("Covariance Matrix $\\Sigma$:", font_size=24, color=YELLOW),
            Matrix([[f"{Sigma[i,j]:.2f}" for j in range(3)] for i in range(3)], h_buff=1.5).scale(0.6)
        ).arrange(DOWN, buff=0.15)
        
        self.play(
            FadeOut(current_text_group), # Fade out previous text
            FadeIn(sigma_tex.to_corner(UR, buff=0.5)), # Place new text
            run_time=short
        )
        current_text_group = sigma_tex # Update for next fade out
        self.add_fixed_orientation_mobjects(current_text_group)
        self.wait(medium)

        # 6. Eigenvalues
        decomp_tex = VGroup(
            Tex("Eigenvalues (Variances along axes):", font_size=24, color=YELLOW),
            MathTex(f"\\lambda_1 = {eigvals[0]:.2f}", color=RED, font_size=24),
            MathTex(f"\\lambda_2 = {eigvals[1]:.2f}", color=GREEN, font_size=24),
            MathTex(f"\\lambda_3 = {eigvals[2]:.2f}", color=BLUE, font_size=24)
        ).arrange(DOWN, aligned_edge=LEFT, buff=0.15)
        
        # Position below the covariance matrix
        decomp_tex.next_to(sigma_tex, DOWN, aligned_edge=LEFT, buff=0.4) 
        
        self.play(
            FadeIn(decomp_tex), 
            run_time=short
        )
        current_text_group.add(decomp_tex) # Add to the current group for combined fade out
        self.add_fixed_orientation_mobjects(current_text_group)
        self.wait(medium)

        # 7. Camera motion
        self.begin_ambient_camera_rotation(rate=0.1)
        self.wait(3)
        self.stop_ambient_camera_rotation()
        self.move_camera(phi=65*DEGREES, theta=45*DEGREES, frame_center=ORIGIN, run_time=long)
        self.wait(medium)

        # 8. Fade out all
        self.play(
            FadeOut(dots),
            FadeOut(ellipsoid_surface),
            FadeOut(eig_arrows),
            FadeOut(mean_arrow),
            FadeOut(current_text_group), # Fade out the final text group
            run_time=medium
        )
        self.wait(short)

%manim -ql -v warning Covariance3D

Manim Community v0.19.0

[11/12/25 19:41:12] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=96085;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=595721;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=253358;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=41872;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

[11/12/25 19:46:03] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=97694;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=728411;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=366257;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=791102;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

...

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=220706;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=23447;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

[11/12/25 20:06:57] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=501657;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=674847;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=937477;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=126431;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

[11/12/25 20:07:30] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=927635;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=732902;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=976225;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=608079;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

In [1]:
from manim import *
import numpy as np

# ------------------------------
# Helper Classes (Unchanged)
# ------------------------------

class Arrow3DSafe(Arrow3D):
    """Stable Arrow3D wrapper for Manim v0.19"""
    def __init__(self, start=ORIGIN, end=RIGHT, color=WHITE, **kwargs):
        super().__init__(
            start=start,
            end=end,
            color=color,
            **{k: v for k,v in kwargs.items() if k not in ["tip_length","thickness","scale_tips"]}
        )

class MatrixTransformHelper:
    @staticmethod
    def apply_matrix_to_vector(M, v):
        v = np.asarray(v).reshape(3,)
        return M.dot(v)


# ------------------------------
# Main Scene (Refactored)
# ------------------------------

class Covariance3D(ThreeDScene):
    def construct(self):
        # Parameters
        mu = np.array([0.5, -0.2, 0.3])
        Sigma = np.array([[1.0, 0.6, 0.2],
                          [0.6, 0.8, 0.1],
                          [0.2, 0.1, 0.5]])
        n_samples = 200  # Reduced for speed
        short, medium, long = 0.5, 1.0, 1.5 # Reduced for speed

        # Eigen decomposition
        eigvals, eigvecs = np.linalg.eigh(Sigma)
        order = np.argsort(eigvals)[::-1]
        eigvals = eigvals[order]
        eigvecs = eigvecs[:, order]
        S = eigvecs.dot(np.diag(np.sqrt(eigvals)))

        # Axes setup for left side
        axes = ThreeDAxes(x_range=[-3,3,1], y_range=[-3,3,1], z_range=[-3,3,1], x_length=6, y_length=6, z_length=6)
        axes.scale(0.8) # Scale down axes
        axes.shift(LEFT*2.5) # Shift to the left
        self.add(axes)
        for direction, axis in zip([RIGHT, UP, OUT], ['x','y','z']):
            label = MathTex(axis, font_size=24)
            label.next_to(axes.get_axis(np.argmax(direction)).get_end(), direction*0.5)
            self.add_fixed_orientation_mobjects(label)

        self.set_camera_orientation(phi=60*DEGREES, theta=-45*DEGREES, distance=12)
        
        # --- Right-Side Text Panel Helper ---
        current_text_group = VGroup()

        def display_text_step(title_text, body_text):
            nonlocal current_text_group
            
            title = Tex(title_text, font_size=28, color=YELLOW)
            body = Tex(body_text, font_size=24, color=WHITE)
            new_text_group = VGroup(title, body).arrange(DOWN, aligned_edge=LEFT, buff=0.2)
            new_text_group.to_corner(UR, buff=0.5)
            
            # The fixed_orientation ensures the text always faces the camera, regardless of 3D rotations
            self.add_fixed_orientation_mobjects(new_text_group) 
            
            anims_to_play = [FadeIn(new_text_group)]
            
            if current_text_group:
                anims_to_play.append(FadeOut(current_text_group))
                
            current_text_group = new_text_group
            
            return anims_to_play


        # --- 3D Scene Animations (Left Side) ---

        # 1. Display Mean Vector
        mean_arrow = Arrow3DSafe(start=ORIGIN, end=mu, color=YELLOW_D, thickness=0.08)
        mean_arrow.shift(LEFT*2.5) # Shift arrow with axes
        
        text_anims_1 = display_text_step("1. Mean Vector ($\\mu$)", "The center of the data distribution.")
        self.play(
            Create(mean_arrow),
            *text_anims_1,
            run_time=medium
        )
        self.wait(short)

        # 2. Gaussian Cloud
        rng = np.random.default_rng(seed=42)
        # Shift samples to align with the left-shifted axes
        samples = rng.multivariate_normal(mean=mu, cov=Sigma, size=n_samples) + LEFT*2.5 
        dots = VGroup()
        for p in samples:
            s = Sphere(radius=0.03)
            s.move_to(p)
            s.set_fill(opacity=0.9)
            s.set_stroke(width=0.02)
            dots.add(s)
            
        text_anims_2 = display_text_step("2. Data Cloud (Samples)", f"{n_samples} samples drawn from a \nmultivariate Gaussian distribution.")
        self.play(
            FadeIn(dots),
            *text_anims_2,
            run_time=long
        )
        self.wait(short)

        # 3. Covariance Ellipsoid
        def ellipsoid_func_shifted(u,v): # Shifted ellipsoid function
            x = np.sin(u)*np.cos(v)
            y = np.sin(u)*np.sin(v)
            z = np.cos(u)
            return np.dot(S, np.array([x,y,z])) + mu + LEFT*2.5

        ellipsoid_surface = Surface(lambda u,v: ellipsoid_func_shifted(u,v),
                                    u_range=[0, PI], v_range=[0, TAU],
                                    resolution=(24,24),
                                    fill_opacity=0.25,
                                    checkerboard_colors=[BLUE_E, BLUE_D])
        
        text_anims_3 = display_text_step("3. Covariance Ellipsoid", "Represents the 1-standard-deviation \nsurface of the distribution's shape.")
        self.play(
            FadeIn(ellipsoid_surface),
            *text_anims_3,
            run_time=medium
        )
        self.wait(short)

        # 4. Eigenvectors (Principal Axes)
        eig_arrows = VGroup()
        colors = [RED, GREEN, BLUE]
        for i in range(3):
            vec = eigvecs[:, i]
            length = np.sqrt(eigvals[i])*1.6 # Scale by sqrt(eigenvalue)
            # Shift eigenvectors with the rest of the 3D scene
            arr = Arrow3DSafe(start=mu-vec*length + LEFT*2.5, end=mu+vec*length + LEFT*2.5, color=colors[i], thickness=0.06)
            eig_arrows.add(arr)

        text_anims_4 = display_text_step("4. Principal Axes (Eigenvectors)", "Directions of maximum variance in the data. \nLengths are proportional to $\\sqrt{\\lambda_i}$.")
        self.play(
            LaggedStartMap(Create, eig_arrows, lag_ratio=0.2),
            *text_anims_4,
            run_time=medium
        )
        self.wait(short)

        # 5. Camera motion for 3D scene
        self.begin_ambient_camera_rotation(rate=0.1)
        self.wait(3)
        self.stop_ambient_camera_rotation()
        # Ensure camera motion only applies to the 3D elements
        self.move_camera(phi=65*DEGREES, theta=45*DEGREES, frame_center=LEFT*2.5, run_time=long) 
        self.wait(medium)

        # 6. Fade out all 3D scene elements and the last text panel
        self.play(
            FadeOut(dots),
            FadeOut(ellipsoid_surface),
            FadeOut(eig_arrows),
            FadeOut(mean_arrow),
            FadeOut(axes), # Also fade out the axes and labels
            FadeOut(current_text_group), # Fade out the final text group
            run_time=medium
        )
        # Ensure that fixed orientation mobjects are removed after fade out.
        # This is implicitly handled when they are faded out, but explicitly removing 
        # the previous text group ensures no lingering references for the matrices part.
        self.remove(*self.get_mobjects_from_fixed_orientation_mobjects()) 
        self.wait(short)

        # --- Matrices (Independent, fixed to camera) ---

        # 1. Covariance Matrix
        sigma_tex_title = Tex("Covariance Matrix $\\Sigma$:", font_size=32, color=YELLOW)
        sigma_matrix = Matrix([[f"{Sigma[i,j]:.2f}" for j in range(3)] for i in range(3)], h_buff=1.5).scale(0.7)
        sigma_group = VGroup(sigma_tex_title, sigma_matrix).arrange(DOWN, buff=0.4)
        sigma_group.to_center().shift(UP*1.5) # Center and shift up

        self.play(FadeIn(sigma_group), run_time=medium)
        self.wait(medium)

        # 2. Eigenvalues
        decomp_tex_title = Tex("Eigenvalues (Variances along axes):", font_size=32, color=YELLOW)
        decomp_values = VGroup(
            MathTex(f"\\lambda_1 = {eigvals[0]:.2f}", color=RED, font_size=30),
            MathTex(f"\\lambda_2 = {eigvals[1]:.2f}", color=GREEN, font_size=30),
            MathTex(f"\\lambda_3 = {eigvals[2]:.2f}", color=BLUE, font_size=30)
        ).arrange(DOWN, aligned_edge=LEFT, buff=0.2)
        
        decomp_group = VGroup(decomp_tex_title, decomp_values).arrange(DOWN, buff=0.4)
        decomp_group.next_to(sigma_group, DOWN, buff=1.0) # Position below covariance matrix

        self.play(FadeIn(decomp_group), run_time=medium)
        self.wait(2)

        # 3. Fade out matrices
        self.play(FadeOut(sigma_group), FadeOut(decomp_group), run_time=medium)
        self.wait(short)

%manim -qk -v warning Covariance3D

Manim Community v0.19.0

[11/13/25 12:02:11] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=107199;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=715528;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=243365;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=601085;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

[11/13/25 12:18:18] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=304481;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=518187;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=660368;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=51909;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

...

[11/13/25 13:42:17] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=605875;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=415357;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=897925;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=746557;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

[11/13/25 13:43:26] WARNING  It looks like the scene contains a lot of sub-mobjects. Caching is      ]8;id=826733;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=51925;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#160\160]8;;\
                             sometimes not suited to handle such large scenes, you might consider                  
                             disabling caching with --disable_caching to potentially speed up the                  
                             rendering process.                                                                    

                    WARNING  You can disable this warning by setting disable_caching_warning to True ]8;id=537731;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py\hashing.py]8;;\:]8;id=857299;file:///home/surya/Desktop/tensor-geometry/.venv/lib/python3.11/site-packages/manim/utils/hashing.py#166\166]8;;\
                             in your config file.                                                                  

AttributeError: 'Covariance3D' object has no attribute 'get_mobjects_from_fixed_orientation_mobjects'